## Imports

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchaudio
from torchaudio.transforms import Resample

import snntorch as snn
from snntorch import spikegen
from snntorch import utils

from sklearn.model_selection import train_test_split

import os

import pandas as pd

import optuna

from tqdm import tqdm

## Config

In [2]:
config = {
    # Filter Data
    "metadata_path": "/home/emercad3/sound_localization/data/metadata.parquet",
    "speech_path": "/home/emercad3/sound_localization/data/speech_ambi",
    #"noise_path": "/home/emercad3/sound_localization/data/noise_ambi",
    "noise_path": None,
    "is_lite_version": True,

    # Load Mix and STFT Audio
    "sr": 16_000,
    "duration": 0.5,
    "noise_ratio": None, # Optional (Large #s make noise louder, Small #s make noise quieter)
    "n_fft": 512,
    "hop_length": 128,
    "win_length": 512,

    # Split Data
    "val_size": 0.1,

    # Dataset
    "num_steps": 40,
    "batch_size": 128,

    # SNN
    "do": 0.6,
    "beta": 0.65,
    "beta_re": 0.65,
    "beta_out": 0.65,
        # Active Branch
    "a_thresh1": 10, "a_thresh2": 10, "a_thresh3": 10, "a_thresh4": 10,
        # Reactive Branch
    "re_thresh1": 10, "re_thresh2": 10, "re_thresh3": 10, "re_thresh4": 10,
        # Output Layer
    "out_thresh1": 10, "out_thresh2": 10,

    # Training
    "num_epochs": 50,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "lr": 0.0003,
    
    # Optuna
    "n_trials": 50,
    "early_stop_epochs": 5,
}

## Preprocessing

### Filter Data

In [3]:
def filter_data(
        metadata_pth=config["metadata_path"],
        speech_pth=config["speech_path"],
        noise_pth=config["noise_path"],
        lite_version=config["is_lite_version"]
):

    metadata = pd.read_parquet(metadata_pth, engine="pyarrow")
    if lite_version:
        metadata = metadata[metadata["lite_version"] == True]

    data = []
    for _, row in metadata.iterrows():
        sample = {
            "sample_id": row["sample_id"],
            "speech_path": os.path.join(speech_pth, f"{row['sample_id']:06}.flac"),
            "noise_path": os.path.join(noise_pth, f"{row['sample_id']:06}.flac") if noise_pth else None,
            "azimuth": row["speech/azimuth"],
            "elevation": row["speech/elevation"],
            "split": row["split"]
        }
        data.append(sample)

    return data

### Split Data

In [4]:
def split_data(data, val_size=config["val_size"]):
    train_data = [sample for sample in data if sample["split"] == "train"]
    test_data = [sample for sample in data if sample["split"] == "test"]

    train_data, val_data = train_test_split(train_data, test_size=val_size, random_state=42)

    return train_data, val_data, test_data


### Load, Trim/Pad, STFT

In [5]:
def load_mix_and_stft_foa_audio(
        speech_pth,
        noise_pth=None,
        sr=config["sr"],
        duration=config["duration"],
        noise_ratio=config["noise_ratio"],
        n_fft=config["n_fft"],
        hop_length=config["hop_length"],
        win_length=config["win_length"],
        device=config["device"]
):

    def preprocess_audio(audio_pth, sr, target_len, device):
        num_frames = target_len
        audio, audio_sr = torchaudio.load(audio_pth, frame_offset=0, num_frames=num_frames)
        audio = audio.to(device)  # Move to GPU
        if audio_sr != sr:
            resample_transform = torchaudio.transforms.Resample(orig_freq=audio_sr, new_freq=sr)
            audio = resample_transform(audio)
        max_val = audio.abs().max()
        if max_val > 0:  # Avoid division by zero
            audio = audio / max_val
        # Check if padding is necessary (after resampling, the actual number of samples might change)
        actual_len = audio.size(1)
        if actual_len < target_len:
            padding_size = target_len - actual_len
            audio = torch.nn.functional.pad(audio, (0, padding_size), "constant", 0)
        return audio

    device = device
    target_len = int(duration * sr)
    speech_audio = preprocess_audio(speech_pth, sr, target_len, device)
    should_renormalize = False

    if noise_pth is not None:
        noise_audio = preprocess_audio(noise_pth, sr, target_len, device)

        if noise_ratio is not None:
            # Adjust noise level relative to speech
            noise_audio = noise_audio * noise_ratio
            should_renormalize = True

        # Mix speech and noise
        mixed_audio = speech_audio + noise_audio
    else:
        mixed_audio = speech_audio

    if should_renormalize:
        # Re-normalize only if noise has been adjusted and mixed
        max_val = mixed_audio.abs().max()
        if max_val > 0:
            mixed_audio = mixed_audio / max_val

    # Move Window to device
    window = torch.hann_window(win_length).to(device)

    # Compute the STFT of the mixed audio
    stft = torch.stft(mixed_audio,
                      n_fft=n_fft,
                      hop_length=hop_length,
                      win_length=win_length,
                      window=window,
                      center=True,
                      normalized=True,
                      onesided=False,
                      return_complex=True)

    return stft


### Calculate Active and Reactive Intensity

In [6]:
def compute_active_reactive_intensities(stft, rho=1.21, c=343):
    """
    Compute active and reactive intensity vectors from STFT of 4-channel FOA audio.
    Args:
    - stft: STFT of the FOA audio with shape [4, Frequency Bins, Time Frames].
    - rho: Mean density of air (in kg/m^3).
    - c: Speed of sound in air (in m/s).

    Returns:
    - Ia: Active intensity vector.
    - Ir: Reactive intensity vector.
    """
    # Constants
    three = torch.tensor(3.0, dtype=torch.float, device=stft.device)
    normalization_factor = -1 / (rho * c * torch.sqrt(three))

    # Extract channels
    p = stft[0]  # Pressure (W channel)
    vx = stft[1] * normalization_factor  # Velocity X
    vy = stft[2] * normalization_factor  # Velocity Y
    vz = stft[3] * normalization_factor  # Velocity Z

    # Compute complex conjugate of pressure
    p_star = torch.conj(p)

    # Calculate active and reactive intensity vectors
    Ia_x = torch.real(p_star * vx)
    Ia_y = torch.real(p_star * vy)
    Ia_z = torch.real(p_star * vz)

    Ir_x = torch.imag(p_star * vx)
    Ir_y = torch.imag(p_star * vy)
    Ir_z = torch.imag(p_star * vz)

    # Create stack for each channel [3, num_samples, num_frames]
    Ia = torch.stack((Ia_x, Ia_y, Ia_z), dim=0)
    Ir = torch.stack((Ir_x, Ir_y, Ir_z), dim=0)

    return Ia, Ir


## Custom Dataset Class

In [7]:
class AmbisonicDataset(Dataset):
    def __init__(self, data, config):
        """
        Args:
            data (list of dicts): Each dictionary contains paths and labels for a sample.
            config (dict): Configuration dictionary including sample rate (sr), duration, etc.
        """
        self.data = data
        self.config = config

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]

        # Load and process ambisonic audio
        speech_path = sample["speech_path"]
        noise_path = sample["noise_path"] if "noise_path" in sample and sample["noise_path"] is not None else None

        stft_audio = load_mix_and_stft_foa_audio(
            speech_path,
            noise_pth=noise_path,
            sr=self.config["sr"],
            duration=self.config["duration"],
            noise_ratio=self.config["noise_ratio"],
            n_fft=self.config["n_fft"],
            hop_length=self.config["hop_length"],
            win_length=self.config["win_length"],
            device=self.config["device"]
        )

        # Compute active and reactive intensities
        Ia, Ir = compute_active_reactive_intensities(stft_audio, rho=1.21, c=343)

        # Generate Spike Trains
        spikes_Ia = spikegen.rate(Ia, num_steps=self.config["num_steps"])
        spikes_Ir = spikegen.rate(Ir, num_steps=self.config["num_steps"])

        azimuth = sample['azimuth']
        elevation = sample['elevation']
        label = torch.tensor([azimuth, elevation], dtype=torch.float)

        #print(f"stft_audio.shape: {stft_audio.shape}\nIa.shape:{Ia.shape}, Ir.shape: {Ir.shape}\nspikes_Ia.shape: {spikes_Ia.shape}, spikes_Ir.shape: {spikes_Ir.shape}\nlabel.shape: {label.shape}")

        return spikes_Ia, spikes_Ir, label


## Model

In [8]:
class SNN(nn.Module):
    def __init__(self, config):
        super(SNN, self).__init__()
        self.num_steps = config["num_steps"]
        self.beta = config["beta"]
        self.beta_re = config["beta_re"]
        self.beta_out = config["beta_out"]

        # Active Thresholds
        self.a_thr1 = config["a_thresh1"]
        self.a_thr2 = config["a_thresh2"]
        self.a_thr3 = config["a_thresh3"]
        self.a_thr4 = config["a_thresh4"]

        # Reactive Thresholds
        self.re_thr1 = config["re_thresh1"]
        self.re_thr2 = config["re_thresh2"]
        self.re_thr3 = config["re_thresh3"]
        self.re_thr4 = config["re_thresh4"]

        ## FC Layer
        self.beta_out = config["beta_out"]
        self.out_thr1 = config["out_thresh1"]
        self.out_thr2 = config["out_thresh2"]
        self.do = config["do"]

        # Define the active branch with LIF neurons
        self.active_branch = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=3),
            nn.BatchNorm2d(6),
            nn.MaxPool2d(2),
            snn.Leaky(beta=self.beta, threshold=self.a_thr1, init_hidden=True),

            nn.Conv2d(6, 12, kernel_size=3),
            nn.BatchNorm2d(12),
            nn.MaxPool2d(2),
            snn.Leaky(beta=self.beta, threshold=self.a_thr2, init_hidden=True),

            nn.Conv2d(12, 24, kernel_size=3),
            nn.BatchNorm2d(24),
            nn.MaxPool2d(2),
            snn.Leaky(beta=self.beta, threshold=self.a_thr3, init_hidden=True),

            nn.Conv2d(24, 48, kernel_size=3),
            nn.BatchNorm2d(48),
            nn.MaxPool2d(2),
            snn.Leaky(beta=self.beta, threshold=self.a_thr4, init_hidden=True),
        )

        self.reactive_branch = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=3),
            nn.BatchNorm2d(6),
            nn.MaxPool2d(2),
            snn.Leaky(beta=self.beta_re, threshold=self.re_thr1, init_hidden=True),

            nn.Conv2d(6, 12, kernel_size=3),
            nn.BatchNorm2d(12),
            nn.MaxPool2d(2),
            snn.Leaky(beta=self.beta_re, threshold=self.re_thr2, init_hidden=True),

            nn.Conv2d(12, 24, kernel_size=3),
            nn.BatchNorm2d(24),
            nn.MaxPool2d(2),
            snn.Leaky(beta=self.beta_re, threshold=self.re_thr3, init_hidden=True),

            nn.Conv2d(24, 48, kernel_size=3),
            nn.BatchNorm2d(48),
            nn.MaxPool2d(2),
            snn.Leaky(beta=self.beta_re, threshold=self.re_thr4, init_hidden=True),
        )

        self.fc = nn.Sequential(
            nn.Linear(5760, 512), # use 4608 when stft onesided=True
            snn.Leaky(beta=self.beta_out, threshold=self.out_thr1, init_hidden=True),
            nn.Linear(512, 256),
            snn.Leaky(beta=self.beta_out, threshold=self.out_thr2, init_hidden=True),
            nn.Linear(256, 128),
            nn.Dropout(p=self.do),
            nn.Linear(128, 2)
        )

    # def forward(self, active_input, reactive_input):
    #     active_in_permuted = active_input.permute(1, 0, 2, 3, 4) # [num_step, batch_size, channels, num_samples, num_frames]
    #     reactive_in_permuted = reactive_input.permute(1, 0, 2, 3, 4)

    #     active_outputs = []
    #     reactive_outputs = []

    #     for step in range(active_in_permuted.size(0)):
    #         active_step = self.active_branch(active_in_permuted[step])
    #         reactive_step = self.reactive_branch(reactive_in_permuted[step])

    #         active_outputs.append(active_step)
    #         reactive_outputs.append(reactive_step)

    #     # Stack the outputs across time steps to form tensors of shape [num_steps, batch_size, channels, height, width]
    #     active_stacked = torch.stack(active_outputs, dim=0)
    #     reactive_stacked = torch.stack(reactive_outputs, dim=0)

    #     # Aggregate across time steps, e.g., by taking the mean or sum
    #     active_agg = torch.mean(active_stacked, dim=0)
    #     reactive_agg = torch.mean(reactive_stacked, dim=0)

    #     # Flatten and concatenate the aggregated outputs for the final MLP
    #     active_flat = active_agg.view(active_agg.size(0), -1)
    #     reactive_flat = reactive_agg.view(reactive_agg.size(0), -1)
    #     combined = torch.cat((active_flat, reactive_flat), dim=1)

    #     output = self.fc(combined)

    #     return output

    def forward(self, active_input, reactive_input):
        step_outputs = []
        permute_active = active_input.permute(1, 0, 2, 3, 4)
        permute_reactive = reactive_input.permute(1, 0, 2, 3, 4)

        if permute_active.size(0) != permute_reactive.size(0):
            raise ValueError("The Tme Steps from active and reactive Do NOT Match")

        for step in range(permute_active.size(0)):

            current_active = permute_active[step]
            current_reactive = permute_reactive[step]

            # Process inputs through active and reactive branches
            active_out = self.active_branch(current_active)
            reactive_out = self.reactive_branch(current_reactive)

            # Flatten and combine the outputs
            combined = torch.cat((active_out, reactive_out), dim=1)
            combined = combined.view(combined.size(0), -1)

            fc_out = self.fc(combined)

            step_outputs.append(fc_out)

        tensor_out = torch.stack(step_outputs, dim=0)
        output = torch.mean(tensor_out, dim=0)

        return output




## Util Functions

In [9]:
def calc_median_absolute_error(t_azimuth, t_elevation, p_azimuth, p_elevation):
    """
    Calculate the median absolute error of the angular distance between the true
    and predicted azimuth and elevation angles.

    Parameters:
    t_azimuth (tensor): Azimuth angles of the true points in radians.
    t_elevation (tensor): Elevation angles of the true points in radians.
    p_azimuth (tensor): Azimuth angles of the predicted points in radians.
    p_elevation (tensor): Elevation angles of the predicted points in radians.

    Returns:
    tensor: The median angular distance in degrees.
    """

    # Calculate the cosine of the angular distance
    cosine_of_angle = (
        torch.sin(t_azimuth) * torch.sin(p_azimuth) +
        torch.cos(t_azimuth) * torch.cos(p_azimuth) * torch.cos(t_elevation - p_elevation)
    )

    # Clamp the cosine of the angle to the range [-1, 1] to avoid errors due to numerical instability
    cosine_of_angle = torch.clamp(cosine_of_angle, -1, 1)

    # Calculate the angular distance in radians
    error_rad = torch.acos(cosine_of_angle)

    # Convert the angular distance from radians to degrees
    error_deg = torch.rad2deg(error_rad)

    # Calculate the median of the absolute errors in degrees
    median_error = torch.median(torch.abs(error_deg))

    return median_error


## Training, Validation and Testing Functions

### Training Function

In [10]:
# def train(model, train_loader, criterion, optimizer, device):
#     model.train()  # Set model to training mode

#     train_loss = 0.0
#     true_azimuths, true_elevations, pred_azimuths, pred_elevations = [], [], [], []

#     for batch_idx, (active_input, reactive_input, labels) in enumerate(tqdm(train_loader, desc="Training")):
#         #print(f"Batch {batch_idx+1}/{len(train_loader)}")

#         # Move data and labels to the device
#         active_input, reactive_input, labels = active_input.to(device), reactive_input.to(device), labels.to(device)

#         # Reset branches - required for init_hidden=True
#         utils.reset(model.active_branch)
#         utils.reset(model.reactive_branch)

#         # Zero the parameter gradients
#         optimizer.zero_grad()

#         # Forward pass
#         outputs = model(active_input, reactive_input)  # Pass both inputs to the model

#         # Compute loss
#         loss = criterion(outputs, labels)

#         # Backward pass and optimize
#         loss.backward()
#         optimizer.step()

#         # Update training loss
#         train_loss = train_loss + loss.item() * active_input.size(0)

#         true_azimuths.append(labels[:, 0].detach())
#         true_elevations.append(labels[:, 1].detach())
#         pred_azimuths.append(outputs[:, 0].detach())
#         pred_elevations.append(outputs[:, 1].detach())

#     # Calculate average loss over the dataset
#     train_loss = train_loss / len(train_loader.dataset)

#     return train_loss, torch.cat(true_azimuths), torch.cat(true_elevations), torch.cat(pred_azimuths), torch.cat(pred_elevations)


In [11]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()  # Set model to training mode

    train_loss = 0.0
    true_azimuths, true_elevations, pred_azimuths, pred_elevations = [], [], [], []

    for batch_idx, (active_input, reactive_input, labels) in enumerate(train_loader):
        # Move data and labels to the device
        active_input, reactive_input, labels = active_input.to(device), reactive_input.to(device), labels.to(device)

        # Check if model is wrapped in DataParallel and access the original model for reset
        if isinstance(model, nn.DataParallel):
            utils.reset(model.module.active_branch)
            utils.reset(model.module.reactive_branch)
        else:
            utils.reset(model.active_branch)
            utils.reset(model.reactive_branch)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(active_input, reactive_input)  # Pass both inputs to the model

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Update training loss
        train_loss += loss.item() * active_input.size(0)

        true_azimuths.append(labels[:, 0].detach())
        true_elevations.append(labels[:, 1].detach())
        pred_azimuths.append(outputs[:, 0].detach())
        pred_elevations.append(outputs[:, 1].detach())

    # Calculate average loss over the dataset
    train_loss = train_loss / len(train_loader.dataset)

    return train_loss, torch.cat(true_azimuths), torch.cat(true_elevations), torch.cat(pred_azimuths), torch.cat(pred_elevations)


### Validation Function

In [12]:
def validate(model, val_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode

    valid_loss = 0.0
    true_azimuths, true_elevations, pred_azimuths, pred_elevations = [], [], [], []

    with torch.no_grad():  # No gradients needed
        for batch_idx, (active_input, reactive_input, labels) in enumerate(val_loader):
            #print(f"Batch {batch_idx+1}/{len(val_loader)}")

            # Move the inputs and labels to the specified device
            active_input, reactive_input, labels = active_input.to(device), reactive_input.to(device), labels.to(device)

            # Forward pass: compute the model output
            outputs = model(active_input, reactive_input)

            # Compute the loss
            loss = criterion(outputs, labels)
            valid_loss = valid_loss + loss.item() * active_input.size(0)

            true_azimuths.append(labels[:, 0].detach())
            true_elevations.append(labels[:, 1].detach())
            pred_azimuths.append(outputs[:, 0].detach())
            pred_elevations.append(outputs[:, 1].detach())

    # Calculate the average loss over the dataset
    valid_loss = valid_loss / len(val_loader.dataset)

    return valid_loss, torch.cat(true_azimuths), torch.cat(true_elevations), torch.cat(pred_azimuths), torch.cat(pred_elevations)


### Testing Function

In [13]:
def test(model, test_loader, criterion, device):
    model.eval()  # Set the model to evaluation mode

    test_loss = 0.0
    true_azimuths, true_elevations, pred_azimuths, pred_elevations = [], [], [], []

    with torch.no_grad():  # No gradients needed during testing
        for batch_idx, (active_input, reactive_input, labels) in enumerate(tqdm(test_loader, desc="Testing")):
            active_input, reactive_input, labels = active_input.to(device), reactive_input.to(device), labels.to(device)

            # Forward pass: compute the model output
            outputs = model(active_input, reactive_input)

            # Compute the loss
            loss = criterion(outputs, labels)
            test_loss = test_loss + loss.item() * active_input.size(0)

            # Optionally, accumulate metrics here
            true_azimuths.append(labels[:, 0].detach())
            true_elevations.append(labels[:, 1].detach())
            pred_azimuths.append(outputs[:, 0].detach())
            pred_elevations.append(outputs[:, 1].detach())

    # Calculate the average loss over the dataset
    test_loss = test_loss / len(test_loader.dataset)

    return test_loss, torch.cat(true_azimuths), torch.cat(true_elevations), torch.cat(pred_azimuths), torch.cat(pred_elevations)


## DataLoaders, Training setup, Training Loop

### DataLoaders

In [14]:
# data = filter_data()
# train_data, val_data, test_data = split_data(data)

# train_dataset = AmbisonicDataset(data=train_data, config=config)
# val_dataset = AmbisonicDataset(data=val_data, config=config)
# test_dataset = AmbisonicDataset(data=test_data, config=config)

# train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, drop_last=True)
# val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False, drop_last=True)
# test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False, drop_last=True)

### Training Setup

In [15]:
# device = config["device"]

# model = SNN(config)
# if torch.cuda.device_count() > 1:
#     print(f"Using {torch.cuda.device_count()} GPUs!")
#     model = torch.nn.DataParallel(model)
# model.to(device)
# criterion = nn.MSELoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=config["lr"])

### Training Loop

In [16]:
# torch.autograd.set_detect_anomaly(True)

# train_loss = []
# valid_loss = []

# for epoch in tqdm(range(config["num_epochs"]), desc="Epochs"):

#     train_epoch_loss, train_true_azimuth, train_true_elevation, train_pred_azimuth, train_pred_elevation = train(model, train_loader, criterion, optimizer, device)
#     valid_epoch_loss, valid_true_azimuth, valid_true_elevation, valid_pred_azimuth, valid_pred_elevation = validate(model, val_loader, criterion, device)

#     train_loss.append(train_epoch_loss)
#     valid_loss.append(valid_epoch_loss)

#     train_angle_error = calc_median_absolute_error(train_true_azimuth, train_true_elevation, train_pred_azimuth, train_pred_elevation)
#     valid_angle_error = calc_median_absolute_error(valid_true_azimuth, valid_true_elevation, valid_pred_azimuth, valid_pred_elevation)

#     print(f"Train Loss: {train_epoch_loss:.4f} | Validation Loss: {valid_epoch_loss:.4f}")
#     print(f"Train Angle Error: {train_angle_error:.4f}° | Validation Angle Error: {valid_angle_error:.4f}°\n")

# test_loss, test_true_azimuth, test_true_elevation, test_pred_azimuth, test_pred_elevation = test(model, test_loader, criterion, device)
# test_angle_error = calc_median_absolute_error(test_true_azimuth, test_true_elevation, test_pred_azimuth, test_pred_elevation)

# print(f"Test Loss: {test_loss:.4f}")
# print(f"Test Angle Error: {test_angle_error:.4f}\n")

## Hypyerparameter Tuning

### Objective Function

In [17]:
# def objective(trial):
#     # Hyperparameters to optimize
#     config["a_thresh1"] = trial.suggest_float("a_thresh1", 1, 50)
#     config["a_thresh2"] = trial.suggest_float("a_thresh2", 1, 50)
#     config["a_thresh3"] = trial.suggest_float("a_thresh3", 1, 50)
#     config["a_thresh4"] = trial.suggest_float("a_thresh4", 1, 50)
#     config["beta"] = trial.suggest_float("beta", 0.1, 0.9)


#     config["re_thresh1"] = trial.suggest_float("re_thresh1", 1, 50)
#     config["re_thresh2"] = trial.suggest_float("re_thresh2", 1, 50)
#     config["re_thresh3"] = trial.suggest_float("re_thresh3", 1, 50)
#     config["re_thresh4"] = trial.suggest_float("re_thresh4", 1, 50)
#     config["beta_re"] = trial.suggest_float("beta_re", 0.1, 0.9)

#     config["out_thresh1"] = trial.suggest_float("out_thresh1", 1, 50)
#     config["out_thresh2"] = trial.suggest_float("out_thresh2", 1, 50)
#     config["beta_out"] = trial.suggest_float("beta_out", 0.1, 0.9)

#     config["do"] = trial.suggest_float("do", 0.01, 0.9)
#     config["num_steps"] = trial.suggest_int("num_steps", 5, 25)
#     config["lr"] = trial.suggest_float("lr", 1e-10, 1e-3)



#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model = SNN(config)
#     if torch.cuda.device_count() > 1:
#         print(f"Using {torch.cuda.device_count()} GPUs.")
#         model = nn.DataParallel(model)

#     model.to(device)
#     criterion = nn.MSELoss()
#     optimizer = torch.optim.AdamW(model.parameters(), lr=config["lr"])

#     data = filter_data()
#     train_data, val_data, _ = split_data(data)

#     train_dataset = AmbisonicDataset(data=train_data, config=config)
#     val_dataset = AmbisonicDataset(data=val_data, config=config)
    

#     train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, drop_last=True)
#     val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False, drop_last=True)

#     for epoch in tqdm(range(config["num_epochs"])):
#         train(model, train_loader, criterion, optimizer, device)
#         val_loss, val_true_azimuth, val_true_elevation, val_pred_azimuth, val_pred_elevation = validate(model, val_loader, criterion, device)
        
#         val_error_deg = calc_median_absolute_error(val_true_azimuth, val_true_elevation, val_pred_azimuth, val_pred_elevation)

#     return val_error_deg.item()  # Optuna minimizes this value

### Objective Function With Early Stopping

In [18]:
def objective(trial):
    # Hyperparameters to optimize
    config["a_thresh1"] = trial.suggest_float("a_thresh1", 1, 50)
    config["a_thresh2"] = trial.suggest_float("a_thresh2", 1, 50)
    config["a_thresh3"] = trial.suggest_float("a_thresh3", 1, 50)
    config["a_thresh4"] = trial.suggest_float("a_thresh4", 1, 50)
    config["beta"] = trial.suggest_float("beta", 0.1, 0.9)

    config["re_thresh1"] = trial.suggest_float("re_thresh1", 1, 50)
    config["re_thresh2"] = trial.suggest_float("re_thresh2", 1, 50)
    config["re_thresh3"] = trial.suggest_float("re_thresh3", 1, 50)
    config["re_thresh4"] = trial.suggest_float("re_thresh4", 1, 50)
    config["beta_re"] = trial.suggest_float("beta_re", 0.1, 0.9)

    config["out_thresh1"] = trial.suggest_float("out_thresh1", 1, 50)
    config["out_thresh2"] = trial.suggest_float("out_thresh2", 1, 50)
    config["beta_out"] = trial.suggest_float("beta_out", 0.1, 0.9)

    config["do"] = trial.suggest_float("do", 0.01, 0.9)
    # config["num_steps"] = trial.suggest_int("num_steps", 5, 40) # 23 is the max num_steps for 2 GPUs
    config["lr"] = trial.suggest_float("lr", 1e-10, 1e-3)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = SNN(config)
    if torch.cuda.device_count() > 1:
        print(f"Using {torch.cuda.device_count()} GPUs.")
        model = nn.DataParallel(model)

    model.to(device)
    criterion = nn.MSELoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=config["lr"])

    data = filter_data()
    train_data, val_data, _ = split_data(data)

    train_dataset = AmbisonicDataset(data=train_data, config=config)
    val_dataset = AmbisonicDataset(data=val_data, config=config)

    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=False, drop_last=True)

    best_val_error_deg = float('inf')
    epochs_no_improve = 0
    early_stop_epochs = config["early_stop_epochs"]

    for epoch in tqdm(range(config["num_epochs"]), desc="Epochs"):
        train(model, train_loader, criterion, optimizer, device)

        _, val_true_azimuth, val_true_elevation, val_pred_azimuth, val_pred_elevation = validate(model, val_loader, criterion, device)
        val_error_deg = calc_median_absolute_error(val_true_azimuth, val_true_elevation, val_pred_azimuth, val_pred_elevation)

        if val_error_deg < best_val_error_deg:
            best_val_error_deg = val_error_deg
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve == early_stop_epochs:
            print("Early stopping triggered.")
            break

    return best_val_error_deg.item()  # Return the best validation angle error



### Run Study Function

In [19]:
def run_study():
    study_name = "snn_study"
    storage_name = "sqlite:///{}.db".format(study_name)

    study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True, direction="minimize")
    study.optimize(objective, n_trials=config["n_trials"])

    # Save the study to disk
    study.trials_dataframe().to_csv("study_results.csv")

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Best trial:")
    trial = study.best_trial
    print("    Value: ", trial.value)
    print("    Params: ")
    for key, value in trial.params.items():
        print(f"      {key}: {value}")



In [20]:
run_study()

[I 2024-03-09 22:13:27,068] A new study created in RDB with name: snn_study


Using 4 GPUs.


Epochs:  46%|▍| 23/5
[I 2024-03-09 23:53:29,382] Trial 0 finished with value: 46.8217658996582 and parameters: {'a_thresh1': 24.14804850839552, 'a_thresh2': 8.705196502743302, 'a_thresh3': 25.457568298566656, 'a_thresh4': 38.89868923524451, 'beta': 0.8364181114137388, 're_thresh1': 39.078790415573735, 're_thresh2': 29.914453168403792, 're_thresh3': 17.961092124810666, 're_thresh4': 18.503767452497325, 'beta_re': 0.5621469668615829, 'out_thresh1': 9.66505877531899, 'out_thresh2': 30.215289591871343, 'beta_out': 0.8877286162285448, 'do': 0.01495168788934992, 'lr': 0.0001246923242269125}. Best is trial 0 with value: 46.8217658996582.


Early stopping triggered.
Using 4 GPUs.


Epochs:  12%| | 6/50
[I 2024-03-10 00:22:16,986] Trial 1 finished with value: 87.75468444824219 and parameters: {'a_thresh1': 18.694148755735057, 'a_thresh2': 32.54513253240053, 'a_thresh3': 42.623932633490966, 'a_thresh4': 33.78888180707709, 'beta': 0.8406655387914931, 're_thresh1': 36.23913190595872, 're_thresh2': 10.535205045366169, 're_thresh3': 38.496165659023, 're_thresh4': 37.18975876129634, 'beta_re': 0.8111615637280576, 'out_thresh1': 18.716781253009863, 'out_thresh2': 23.375438396962288, 'beta_out': 0.3676250543616044, 'do': 0.14917582538959862, 'lr': 0.0006091240821915014}. Best is trial 0 with value: 46.8217658996582.


...

Epochs:  46%|▍| 23/5
[I 2024-03-12 15:35:47,359] Trial 46 finished with value: 45.69802474975586 and parameters: {'a_thresh1': 42.13826980107903, 'a_thresh2': 23.559660127598193, 'a_thresh3': 36.26198444708266, 'a_thresh4': 19.44481689636575, 'beta': 0.5453161994978813, 're_thresh1': 25.837315099475717, 're_thresh2': 49.880675799283495, 're_thresh3': 15.104282902935289, 're_thresh4': 49.88219244161923, 'beta_re': 0.31509665855934743, 'out_thresh1': 38.28019602568013, 'out_thresh2': 8.976403264528662, 'beta_out': 0.6791633115303046, 'do': 0.3769985775447732, 'lr': 0.0008073803705156071}. Best is trial 36 with value: 32.09950637817383.


Early stopping triggered.
Using 4 GPUs.


Epochs:  46%|▍| 23/5
[I 2024-03-12 17:14:44,158] Trial 47 finished with value: 43.43788146972656 and parameters: {'a_thresh1': 47.281999669785364, 'a_thresh2': 22.20891766453178, 'a_thresh3': 39.25762339952405, 'a_thresh4': 13.029488675293893, 'beta': 0.6025089679129183, 're_thresh1': 27.834788806784587, 're_thresh2': 41.321576142637625, 're_thresh3': 10.524677736442207, 're_thresh4': 46.63183325258827, 'beta_re': 0.32055718065944055, 'out_thresh1': 30.549286784421817, 'out_thresh2': 3.6656110001545983, 'beta_out': 0.8940690488152928, 'do': 0.4316356640096315, 'lr': 0.0007224862522925233}. Best is trial 36 with value: 32.09950637817383.


Early stopping triggered.
Using 4 GPUs.


Epochs:   8%| | 4/50